# Food Delivery Time Prediction Model

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import plotly.express as px

data=pd.read_csv("deliverytime.txt")
print(data.head())

     ID Delivery_person_ID  Delivery_person_Age  Delivery_person_Ratings  \
0  4607     INDORES13DEL02                   37                      4.9   
1  B379     BANGRES18DEL02                   34                      4.5   
2  5D6D     BANGRES19DEL01                   23                      4.4   
3  7A6A    COIMBRES13DEL02                   38                      4.7   
4  70A2     CHENRES12DEL01                   32                      4.6   

   Restaurant_latitude  Restaurant_longitude  Delivery_location_latitude  \
0            22.745049             75.892471                   22.765049   
1            12.913041             77.683237                   13.043041   
2            12.914264             77.678400                   12.924264   
3            11.003669             76.976494                   11.053669   
4            12.972793             80.249982                   13.012793   

   Delivery_location_longitude Type_of_order Type_of_vehicle  Time_taken(min)  
0     

In [4]:
R=6371


def deg_to_rad(degrees):
    return degrees * (np.pi/180)


def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  

data['distance'] = np.nan

for i in range(len(data)):
    data.loc[i, 'distance'] = distcalculate(data.loc[i, 'Restaurant_latitude'], 
                                        data.loc[i, 'Restaurant_longitude'], 
                                        data.loc[i, 'Delivery_location_latitude'], 
                                        data.loc[i, 'Delivery_location_longitude'])

In [5]:
x=np.array(data[["Delivery_person_Age",
                "Delivery_person_Ratings",
                "distance"]])
y=np.array(data[["Time_taken(min)"]])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10, random_state=42)

In [6]:
from keras.models import Sequential
from keras.layers import Dense,LSTM

model=Sequential()
model.add(LSTM(128,return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

c:\Users\aarku\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 3, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=9)

Epoch 1/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 215s 5ms/step - loss: 76.1513
Epoch 2/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 183s 4ms/step - loss: 63.8677
Epoch 3/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 194s 5ms/step - loss: 60.8424
Epoch 4/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 149s 4ms/step - loss: 60.5541
Epoch 5/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 157s 4ms/step - loss: 60.5632
Epoch 6/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 159s 4ms/step - loss: 59.1576
Epoch 7/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 58.5767
Epoch 8/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 140s 3ms/step - loss: 59.1663
Epoch 9/9
41033/41033 ━━━━━━━━━━━━━━━━━━━━ 140s 3ms/step - loss: 58.3761


In [8]:
print("Food Delivery Time Prediction")
a=int(input("Age Of Delivery Partner:"))
b=float(input("Ratings of Previous Deliveries: "))
c=int(input("Total Distance: "))

features=np.array([[a,b,c]])
print("Predicted Delivery Time in Minutes= ",model.predict(features))

Food Delivery Time Prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
Predicted Delivery Time in Minutes=  [[29.99002]]
